<a href="https://colab.research.google.com/github/patrickbryant1/MoLPC/blob/master/MoLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Install dependencies
%pip install biopython
%pip install JAX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
!git clone https://github.com/patrickbryant1/MoLPC.git

fatal: destination path 'MoLPC' already exists and is not an empty directory.


In [16]:
#Run the pipeline
#@title Default title text
ID = "1A8R" #@param {type:"string"}
SUBSIZE = 3 #@param {type:"integer"} #What order the subcomplexes should be (2 or 3)
DATADIR="./data/test/"
USEQS=DATADIR+ID+'_useqs.csv'
CHAINS=DATADIR+ID+'_chains.csv'
INTERACTIONS='' #Leave empty if the interactions are not known - here they are not used. See the file $DATADIR/$ID'_ints.csv' to how to supply such a file
GET_ALL=1 #If to get all interactions (1) or not (0) - when the interactions are known

In [61]:
#Write the Paired and Block Diagonalized MSAs to predict sub-components
MSADIR=DATADIR+'/hhblits'
import sys
from google.colab import files
sys.path.insert(0,'/content/MoLPC/src')



In [66]:
from preprocess import preprocess_colab

ImportError: ignored

ModuleNotFoundError: ignored